# SWMM database
Create and export hydraulic head and runoff simulations for synthetic or real rainfalls

## Imports and directories

Imports

In [1]:
import pickle
import numpy as np

import plotly.graph_objects as go

#Custom utils
import utils_data as ud

Configuration information

In [2]:
config_info = ud.import_config_from_yaml('database_config.yaml')

#Directories
rainfall_dats_directory = config_info['rainfall_dats_directory']

training_rainfall_dats_directory    =   rainfall_dats_directory + '/training'
validation_rainfall_dats_directory  =   rainfall_dats_directory + '/validation'
testing_rainfall_dats_directory     =   rainfall_dats_directory + '/testing'


simulations_path = config_info['simulations_directory']

training_simulations_path   = simulations_path + '/training'
validation_simulations_path = simulations_path + '/validation'
testing_simulations_path    = simulations_path + '/testing'


inp_path = config_info['inp_directory']

#Variables
n_synthetic_rainfalls = config_info['n_synthetic_rainfalls']
block_values = config_info['block_values']
block_durations = config_info['block_durations']

n_real_rains = config_info['n_real_rains']

## Synthetic rainfalls

1. Create synthetic rainfalls

Block rainfalls

In [3]:
block_rain = ud.rain_blocks(block_values, block_durations, dt=5)

In [4]:
hietograph_fig = ud.hietograph(block_rain, 'Synthetic hietograph')
hietograph_fig.show()

In [5]:
ud.create_datfiles([block_rain], rainfall_dats_directory, identifier = 'block', isReal=False)

Alternating blocks method

In [6]:
alternating_random_rainfalls = ud.get_multiple_alt_blocks_rainfalls(n_synthetic_rainfalls)

In [7]:
hietograph_fig = ud.hietograph(alternating_random_rainfalls[9], 'Synthetic hietograph')
hietograph_fig.show()

In [8]:
ud.create_datfiles(alternating_random_rainfalls, rainfall_dats_directory, identifier = 'synt', isReal=False)

## Real rainfalls

In [9]:
# %% Read rainfall in each pixel
with open('.\\real_rainfalls\\events_pixel1_2014_5h.pk', 'rb') as f:
  pixel1 = pickle.load(f)
with open('.\\real_rainfalls\\events_pixel2_2014_5h.pk', 'rb') as f:
  pixel2 = pickle.load(f)
with open('.\\real_rainfalls\\events_pixel3_2014_5h.pk', 'rb') as f:
  pixel3 = pickle.load(f)

In [10]:
fig = go.Figure()

rain = pixel2[12]
fig=ud.hietograph(rain, 'Real rainfall hietograph')
fig.show()

In [11]:
rains_larger_than_ten = []

i =0
while len(rains_larger_than_ten)<n_real_rains or i<=100:
    
    try:
        max_rain = ud.get_max_from_raindict(pixel2[i])
    except Exception as e:
        max_rain = 0
        print('error with rain #'+str(i))
    
    if max_rain > 10:
        rains_larger_than_ten.append(pixel2[i])

    i+=1


error with rain #54
error with rain #57
error with rain #65
error with rain #67
error with rain #79
error with rain #96
error with rain #113
error with rain #139
error with rain #154
error with rain #157
error with rain #175
error with rain #176
error with rain #183
error with rain #185


In [12]:
n_real_rains

100

In [13]:
ud.create_datfiles(rains_larger_than_ten, rainfall_dats_directory, identifier = 'real', isReal=True)

## Run SWMM 

In [15]:
ud.run_SWMM(inp_path, training_rainfall_dats_directory, training_simulations_path)

In [17]:
ud.run_SWMM(inp_path, validation_rainfall_dats_directory, validation_simulations_path)

In [19]:
ud.run_SWMM(inp_path, testing_rainfall_dats_directory, testing_simulations_path)

## Extract and export SWMM results

In [16]:
#Hydraulic head and subcatchment runoff
ud.extract_and_pickle_SWMM_results(training_simulations_path)

In [18]:
ud.extract_and_pickle_SWMM_results(validation_simulations_path)

In [20]:
ud.extract_and_pickle_SWMM_results(testing_simulations_path)